# Install a few dependencies

In [1]:
!conda install -y -c conda-forge onnx

Solving environment: done

## Package Plan ##

  environment location: /home/ec2-user/anaconda3/envs/mxnet_p36

  added / updated specs: 
    - onnx


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    onnx-1.0.1                 |           py36_1         4.4 MB  conda-forge

The following NEW packages will be INSTALLED:

    onnx:            1.0.1-py36_1         conda-forge

The following packages will be UPDATED:

    ca-certificates: 2018.8.24-ha4d7672_0 conda-forge --> 2018.10.15-ha4d7672_0 conda-forge
    certifi:         2018.8.24-py36_1     conda-forge --> 2018.10.15-py36_1000  conda-forge
    openssl:         1.0.2p-h470a237_0    conda-forge --> 1.0.2p-h470a237_1     conda-forge


onnx-1.0.1           | 4.4 MB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [2]:
!pip install mxnet-model-server

    100% |████████████████████████████████| 942kB 5.1MB/s ta 0:00:011
    100% |████████████████████████████████| 66.2MB 240kB/s eta 0:00:01    22% |███████▎                        | 15.1MB 8.9MB/s eta 0:00:06    50% |████████████████▎               | 33.7MB 9.5MB/s eta 0:00:04    59% |███████████████████             | 39.1MB 9.3MB/s eta 0:00:03    67% |█████████████████████▊          | 44.9MB 9.1MB/s eta 0:00:03    74% |███████████████████████▉        | 49.2MB 8.6MB/s eta 0:00:02    77% |████████████████████████▊       | 51.1MB 8.4MB/s eta 0:00:02
  Running setup.py bdist_wheel for onnx ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/fb/13/8e/b5652eb574597bfed05b000ed359413029ef2391f24315b733
Successfully built onnx
keras 2.2.2 has requirement keras-preprocessing==1.0.2, but you'll have keras-preprocessing 1.0.1 which is incompatible.
  Found existing installation: onnx 1.2.1
    Uninstalling onnx-1.2.1:
      Successfully uninstalled onnx-1.2.1
You are using pip vers

# Downloading and verifying an ONNX model

In [3]:
!mkdir squeezenet
%cd squeezenet
!curl -O https://s3.amazonaws.com/model-server/models/onnx-squeezenet/squeezenet.onnx

/home/ec2-user/SageMaker/squeezenet
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4836k  100 4836k    0     0  12.9M      0 --:--:-- --:--:-- --:--:-- 12.9M


In [4]:
import onnx

# Load the ONNX model
model = onnx.load("squeezenet.onnx")

# Check that the IR is well formed, identified issues will error out
onnx.checker.check_model(model)

# Create an MXNet Model Archive bundle for serving

In [5]:
# Let's go ahead and download the files we need:
!curl -O https://s3.amazonaws.com/model-server/models/onnx-squeezenet/signature.json
!curl -O https://s3.amazonaws.com/model-server/models/onnx-squeezenet/synset.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   259  100   259    0     0   2670      0 --:--:-- --:--:-- --:--:--  2642
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 31675  100 31675    0     0   236k      0 --:--:-- --:--:-- --:--:--  236k


In [6]:
# Let's take a peek into the **signature.json** file:
!cat signature.json

{
  "inputs": [
    {
      "data_name": "input_0",
      "data_shape": [0, 3, 224, 224]
    }
  ],
  "input_type": "image/jpeg",
  "outputs": [
    {
      "data_name": "softmax",
      "data_shape": [0, 1000]
    }
  ],
  "output_type": "application/json"
}

In [7]:
# Let's take a peek into the synset.txt file:
!head -n 10 synset.txt

n01440764 tench, Tinca tinca
n01443537 goldfish, Carassius auratus
n01484850 great white shark, white shark, man-eater, man-eating shark, Carcharodon carcharias
n01491361 tiger shark, Galeocerdo cuvieri
n01494475 hammerhead, hammerhead shark
n01496331 electric ray, crampfish, numbfish, torpedo
n01498041 stingray
n01514668 cock
n01514859 hen
n01518878 ostrich, Struthio camelus


In [8]:
# Let's package everything up into a Model Archive bundle
!mxnet-model-export --model-name squeezenet --model-path .
!ls -l squeezenet.model

-rw-rw-r-- 1 ec2-user ec2-user 5008808 Nov  7 15:22 squeezenet.model


# Serving the model with MXNet model server

In [9]:
# Spawning a new process to run the server
import subprocess as sp
server = sp.Popen("mxnet-model-server --models squeezenet=squeezenet.model", shell=True)

In [11]:
# Check out the health endpoint
!curl http://127.0.0.1:8080/ping

{
  "health": "healthy!"
}


In [14]:
# Download an image Trying out image classification
!curl -O https://s3.amazonaws.com/model-server/inputs/kitten.jpg
!curl -X POST http://127.0.0.1:8080/squeezenet/predict -F "data_0=@kitten.jpg"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  108k  100  108k    0     0   866k      0 --:--:-- --:--:-- --:--:--  866k
{
  "prediction": [
    [
      {
        "class": "n02124075 Egyptian cat", 
        "probability": 0.6482629179954529
      }, 
      {
        "class": "n02123045 tabby, tabby cat", 
        "probability": 0.2318260818719864
      }, 
      {
        "class": "n02123159 tiger cat", 
        "probability": 0.10045160353183746
      }, 
      {
        "class": "n02127052 lynx, catamount", 
        "probability": 0.013487360440194607
      }, 
      {
        "class": "n02128757 snow leopard, ounce, Panthera uncia", 
        "probability": 0.003135664388537407
      }
    ]
  ]
}


# Clean up

In [ ]:
# Lastly, we'll terminate the server
server.terminate()